In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# PREPROCESSING

In [ ]:
img_size=256
BATCH_size=32
channels=3

In [ ]:
import os
import cv2
import numpy as np
BASE_PATH = '/kaggle/input/ultrasound-breast-images-for-breast-cancer/ultrasound breast classification/train'
unique_classes = os.listdir(BASE_PATH)
class_index = [unique_classes[0], unique_classes[1]] 

In [ ]:
import tensorflow as tf
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/ultrasound-breast-images-for-breast-cancer/ultrasound breast classification/train", 
    shuffle=True,
    image_size=(img_size, img_size),
    batch_size=BATCH_size
)

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/ultrasound-breast-images-for-breast-cancer/ultrasound breast classification/val", 
    shuffle=True,
    image_size=(img_size, img_size),
    batch_size=BATCH_size
)

In [ ]:
class_names=dataset.class_names
class_names

In [ ]:
len(dataset)

In [ ]:
254*32

In [ ]:
import matplotlib.pyplot as plt
for img_batch,label_batch in dataset.take(1):
    for i in range(12):
        plt.subplot(3,4,i+1)
        plt.imshow(img_batch[i].numpy().astype("uint8"))
        plt.axis("off")
        plt.title(class_names[label_batch[i]])
        #print(label_batch.numpy())

In [ ]:
train_size=0.9
len(dataset)*train_size

In [ ]:
train_ds= dataset.take(228)
len(train_ds)

In [ ]:
val_ds=dataset.skip(228) 
len(val_ds)

In [ ]:
def get_dataset_partitions_tf(ds, train_split=0.9, val_split=0.1, shuffle=True, shuffle_size=10000):
    ds_size = len(ds)

    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)

    # Calculate the number of training samples
    train_size = int(train_split * ds_size)

    # Calculate the number of validation samples
    val_size = ds_size - train_size  # Allocate the rest to validation

    # Create the training dataset
    train_ds = ds.take(train_size)

    # Create the validation dataset
    val_ds = ds.skip(train_size).take(val_size)

    return train_ds, val_ds


In [ ]:
train_ds,val_ds=get_dataset_partitions_tf(dataset)

In [ ]:
len(train_ds)

In [ ]:
len(val_ds)

In [ ]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(img_size, img_size),
    layers.Rescaling(1.0 / 255)
])

In [ ]:
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(img_size, img_size),
    layers.Rescaling(1.0 / 255)
])

In [ ]:
data_aug=tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2)
    
])

# CNN MODEL

In [ ]:
from tensorflow.keras import layers, models

# Define your input shape without the batch size
input_shape = (img_size, img_size, channels)
n_classes = 2

# Assuming resize_and_rescale and data_aug are preprocessing layers
model = models.Sequential([
    layers.InputLayer(shape=input_shape),  # Use `shape` instead of `input_shape`
    resize_and_rescale,  # Preprocessing layer
    data_aug,            # Data augmentation layer
    layers.Conv2D(32, (3, 3), activation='relu', name="conv2d_1"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', name="conv2d_2"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', name="conv2d_3"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', name="conv2d_4"),  # Verify layer name
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', name="conv2d_5"),  # Last Conv layer
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

# No need to call model.build() since input shape is defined

In [ ]:
input_shape

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping


early_stopping = EarlyStopping(
    monitor='val_loss',           
    patience=5,                   
    restore_best_weights=True     
)

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
    metrics=['accuracy'] 
)

In [ ]:
EPOCHS = 30
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_size,       
    verbose=1,                   
    validation_data=val_ds
)

# CNN ANALYSIS

In [ ]:
scores=model.evaluate(test_ds)

In [ ]:
scores

In [ ]:
history.params

In [ ]:
history.history.keys()

In [ ]:
CNN_train_acc=history.history['accuracy']
print("Training Accuracy:", CNN_train_acc[-1])

In [ ]:
CNN_val_acc = history.history['val_accuracy']
print("Validation Accuracy:", CNN_val_acc[-1])

In [ ]:
cnn_acc= history.history['accuracy']
cnn_val_acc= history.history['val_accuracy']
cnn_loss= history.history['loss']
cnn_val_loss= history.history['val_loss']

In [ ]:
# Get the actual number of epochs run
actual_epochs = len(history.history['loss'])

# Plot Training and Validation Accuracy and Loss
plt.figure(figsize=(14, 6))

# Accuracy Plot
plt.subplot(1, 2, 1)
plt.plot(range(actual_epochs), history.history['accuracy'], label='Training Accuracy', marker='o', markevery=5, linewidth=1.5)
plt.plot(range(actual_epochs), history.history['val_accuracy'], label='Validation Accuracy', marker='o', markevery=5, linewidth=1.5)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Training and Validation Accuracy', fontsize=14)
plt.legend(loc='lower right', fontsize=10)

# Loss Plot
plt.subplot(1, 2, 2)
plt.plot(range(actual_epochs), history.history['loss'], label='Training Loss', marker='o', markevery=5, linewidth=1.5)
plt.plot(range(actual_epochs), history.history['val_loss'], label='Validation Loss', marker='o', markevery=5, linewidth=1.5)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Training and Validation Loss', fontsize=14)
plt.legend(loc='upper right', fontsize=10)

# Adjust layout and show the plot
plt.tight_layout()
plt.show()


In [ ]:
# Evaluate accuracy on the validation set
val_loss, val_accuracy = model.evaluate(val_ds)
print(f"Validation Accuracy: {val_accuracy:.2f}")


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Extract true labels and predictions from the validation set
y_true = []
y_pred = []

for images, labels in val_ds:
    predictions = model.predict(images)
    y_true.extend(labels.numpy())                   # True labels
    y_pred.extend(np.argmax(predictions, axis=1))  # Predicted class indices

# Generate classification report
report = classification_report(y_true, y_pred, target_names=class_names)
print(report)


In [ ]:
# Evaluate model and print validation metrics
val_metrics = model.evaluate(val_ds, return_dict=True)
print("Validation Metrics:")
for metric, value in val_metrics.items():
    print(f"{metric}: {value:.2f}")


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import tensorflow as tf
from tensorflow.keras import models, layers
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)  # Get predicted probabilities
    y_true.extend(labels.numpy())   # True labels
    y_pred.extend(preds[:, 1])      # Get the probabilities for the positive class

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the ROC curve
fpr, tpr, thresholds = roc_curve(y_true, y_pred)
roc_auc = auc(fpr, tpr)

# Plotting the ROC curve
plt.figure(figsize=(8, 5))
plt.plot(fpr, tpr, color='blue', label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='red', linestyle='--')  # Diagonal line (chance level)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid()
plt.show()

# RESNET

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
resnet_model= Sequential()

pretrained_model=tf.keras.applications.ResNet50(include_top=False,
                                               input_shape=(256,256,3),
                                               pooling='avg',
                                               classes=2,
                                               weights='imagenet')
for layer in pretrained_model.layers: 
    layer.trainable = False
resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512,activation='relu'))
resnet_model.add(Dense(2,activation='softmax'))


In [ ]:
resnet_model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping


early_stopping = EarlyStopping(
    monitor='val_loss',           
    patience=5,                   
    restore_best_weights=True     
)

In [ ]:
from tensorflow.keras.optimizers import Adam

resnet_model.compile(optimizer=Adam(),
                     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                     metrics=['accuracy'])

In [ ]:
EPOCHS=50
history_resnet = resnet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[early_stopping] 
)

# RESNET ANALYSIS

In [ ]:
scores_resnet=resnet_model.evaluate(test_ds)

In [ ]:
scores_resnet


In [ ]:
history_resnet.params

In [ ]:
history_resnet.history['accuracy']

In [ ]:
Resnet_train_acc=history_resnet.history['accuracy']
print("Training Accuracy:", Resnet_train_acc[-1])


In [ ]:
Resnet_val_acc= history_resnet.history['val_accuracy']
print("Validation Accuracy:", Resnet_val_acc[-1])

In [ ]:
resnet_acc= history_resnet.history['accuracy']
resnet_val_acc= history_resnet.history['val_accuracy']
resnet_loss= history_resnet.history['loss']
resnet_val_loss= history_resnet.history['val_loss']

In [ ]:
# Get the number of epochs actually run (it will be less than or equal to EPOCHS)
epochs_run = len(history_resnet.history['accuracy'])

# Plot Training and Validation Accuracy
plt.figure(figsize=(9, 4))
plt.subplot(1, 2, 1)
plt.plot(range(epochs_run), history_resnet.history['accuracy'], label='Training Accuracy',marker='o')
plt.plot(range(epochs_run), history_resnet.history['val_accuracy'], label='Validation Accuracy',marker='o')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# Plot Training and Validation Loss

plt.subplot(1, 2, 2)
plt.plot(range(epochs_run), history_resnet.history['loss'], label='Training Loss',marker='o')
plt.plot(range(epochs_run), history_resnet.history['val_loss'], label='Validation Loss',marker='o')
plt.legend(loc='lower right')
plt.title('Training and Validation Loss')


In [ ]:
# Evaluate accuracy on the validation set
val_loss, val_accuracy = resnet_model.evaluate(val_ds)
print(f"Validation Accuracy: {val_accuracy:.2f}")


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Extract true labels and predictions from the validation set
y_true = []
y_pred = []

for images, labels in val_ds:
    predictions = resnet_model.predict(images)
    y_true.extend(labels.numpy())                   # True labels
    y_pred.extend(np.argmax(predictions, axis=1))  # Predicted class indices

# Generate classification report
report = classification_report(y_true, y_pred, target_names=class_names)
print(report)


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import tensorflow as tf
from tensorflow.keras import models, layers
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = resnet_model.predict(images)  # Get predicted probabilities
    y_true.extend(labels.numpy())   # True labels
    y_pred.extend(preds[:, 1])      # Get the probabilities for the positive class

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the ROC curve
fpr, tpr, thresholds = roc_curve(y_true, y_pred)
roc_auc = auc(fpr, tpr)

# Plotting the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='red', linestyle='--')  # Diagonal line (chance level)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid()
plt.show()

In [ ]:
# Evaluate model and print validation metrics
val_metrics = resnet_model.evaluate(val_ds, return_dict=True)
print("Validation Metrics:")
for metric, value in val_metrics.items():
    print(f"{metric}: {value:.2f}")


# EFFICIENT NET

In [ ]:
!pip install efficientnet


In [ ]:
from tensorflow.keras.applications import EfficientNetB0
import efficientnet.tfkeras as efn

base_model = efn.EfficientNetB0(input_shape = (256, 256, 3), include_top = False, weights = 'imagenet')


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
predictions = Dense(1, activation="sigmoid")(x)
model_efficientNet = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
from tensorflow.keras.optimizers import RMSprop
model_efficientNet.compile(optimizer=RMSprop(learning_rate=0.0001, decay=1e-6), 
                           loss='binary_crossentropy', 
                           metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',          
    patience=5,                  
    restore_best_weights=True     
)

In [ ]:
history_efficientNet = model_efficientNet.fit(
    train_ds,
    validation_data=val_ds,
    steps_per_epoch=100,
    epochs=EPOCHS,
    callbacks=[early_stopping] 
)

# ANALYSIS OF EFFICIENT NET

In [ ]:
scores_efficientNet=model_efficientNet.evaluate(test_ds)

In [ ]:
scores_efficientNet

In [ ]:
history_efficientNet.params

In [ ]:
history_efficientNet.history['accuracy']

In [ ]:
Eff_train_acc = history_efficientNet.history['accuracy']
print("Training Accuracy:", Eff_train_acc[-1])

In [ ]:
Eff_val_acc = history_efficientNet.history['val_accuracy']
print("Validation Accuracy:", Eff_val_acc[-1])

In [ ]:
efficientNet_acc= history_efficientNet.history['accuracy']
efficientNet_val_acc= history_efficientNet.history['val_accuracy']
efficientNet_loss= history_efficientNet.history['loss']
efficientNet_val_loss= history_efficientNet.history['val_loss']

In [ ]:
# Determine the number of epochs actually run
epochs_run = len(efficientNet_acc)

# Plot Training and Validation Accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(range(epochs_run), efficientNet_acc, label='Training Accuracy', marker='o')
plt.plot(range(epochs_run), efficientNet_val_acc, label='Validation Accuracy', marker='o')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# Plot Training and Validation Loss
plt.subplot(1, 2, 2)
plt.plot(range(epochs_run), efficientNet_loss, label='Training Loss', marker='o')
plt.plot(range(epochs_run), efficientNet_val_loss, label='Validation Loss', marker='o')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

# Adjust layout and show the plots
plt.tight_layout()
plt.show()


In [ ]:
# Evaluate accuracy on the validation set
val_loss, val_accuracy = model_efficientNet.evaluate(val_ds)
print(f"Validation Accuracy: {val_accuracy:.2f}")


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Extract true labels and predictions from the validation set
y_true = []
y_pred = []

for images, labels in val_ds:
    predictions = model_efficientNet.predict(images)
    y_true.extend(labels.numpy())                   # True labels
    y_pred.extend(np.argmax(predictions, axis=1))  # Predicted class indices

# Generate classification report
report = classification_report(y_true, y_pred, target_names=class_names)
print(report)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

def plot_roc_curve(model, test_ds):
    y_true = []
    y_pred = []
    
    # Get predictions and true labels
    for images, labels in test_ds:
        preds = model_efficientNet.predict(images)         # Model predictions
        y_true.extend(labels.numpy())         # True labels
        y_pred.extend(preds.flatten())        # Flatten predictions for positive class
    
    # Convert lists to numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Calculate ROC curve
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)
    
    # Plot the ROC curve
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='blue', label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='red', linestyle='--')  # Diagonal line (chance level)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.grid()
    plt.show()

# Usage Example:
# Assuming `model` is your trained model and `test_ds` is your test dataset
plot_roc_curve(model, test_ds)


In [ ]:
# Evaluate model and print validation metrics
val_metrics = model_efficientNet.evaluate(val_ds, return_dict=True)
print("Validation Metrics:")
for metric, value in val_metrics.items():
    print(f"{metric}: {value:.2f}")


# COMPARATIVE STUDY

In [ ]:
import matplotlib.pyplot as plt

# Evaluate models and extract accuracy
scores = model.evaluate(test_ds)
scores_resnet = resnet_model.evaluate(test_ds)
scores_efficientNet = model_efficientNet.evaluate(test_ds)

# Assuming the accuracy is the second value in the `scores` array
accuracy_model = scores[1]
accuracy_resnet = scores_resnet[1]
accuracy_efficientNet = scores_efficientNet[1]

# Create a list of model names and corresponding accuracies
model_names = ['Base Model', 'ResNet', 'EfficientNet']
accuracies = [accuracy_model, accuracy_resnet, accuracy_efficientNet]

# Plotting the accuracies on a bar graph
plt.figure(figsize=(6, 4))  # Decreased figure size
plt.bar(model_names, accuracies, color=['blue', 'green', 'orange'], width=0.4)  # Reduced bar width

# Adding titles and labels
plt.title('Test Accuracy Comparison of Models', fontsize=12)
plt.xlabel('Models', fontsize=10)
plt.ylabel('Test Accuracy', fontsize=10)

# Show the plot
plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Example accuracy and validation accuracy for the models
# Replace these with your actual values
models = ['CNN', 'ResNet', 'EfficientNet']
# Assuming CNN_train_acc, Resnet_train_acc, Eff_train_acc are lists with accuracy per epoch
# Use the last value (or final accuracy) for each model
accuracy = [CNN_train_acc[-1], Resnet_train_acc[-1], Eff_train_acc[-1]]  # Final Training accuracy for CNN, ResNet, and EfficientNet
val_accuracy = [CNN_val_acc[-1], Resnet_val_acc[-1], Eff_val_acc[-1]]  # Final Validation accuracy for CNN, ResNet, and EfficientNet

# Set Seaborn style and customize the background
sns.set_style("whitegrid")
plt.rcParams['axes.facecolor'] = '#f7f7f7'  # Off-white background color

# Setting up the bar positions
x = np.arange(len(models))  # the label locations
width = 0.2  # Smaller bar width

# Create the bar chart
fig, ax = plt.subplots(figsize=(6, 4))  # Smaller figure size
bars1 = ax.bar(x - width/2, accuracy, width, label='Training Accuracy', color='black')
bars2 = ax.bar(x + width/2, val_accuracy, width, label='Validation Accuracy', color='red')

# Add labels, title, and legend
ax.set_xlabel('Models', fontsize=12)
ax.set_ylabel('Accuracy', fontsize=12)
ax.set_title('Comparative Study of Model Accuracy', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(models, fontsize=10)
ax.legend(fontsize=10, loc='lower right')

# Annotate bars with values
for bar in bars1:
    height = bar.get_height()
    ax.annotate(f'{height:.2f}', 
                xy=(bar.get_x() + bar.get_width() / 2, height), 
                xytext=(0, 3),  # Offset by 3 points
                textcoords="offset points",
                ha='center', va='bottom', fontsize=8)

for bar in bars2:
    height = bar.get_height()
    ax.annotate(f'{height:.2f}', 
                xy=(bar.get_x() + bar.get_width() / 2, height), 
                xytext=(0, 3), 
                textcoords="offset points",
                ha='center', va='bottom', fontsize=8)

# Adjust layout and show plot
plt.tight_layout()
plt.show()


In [ ]:
from tensorflow.keras.models import load_model

# Correct model path
model_path = "/kaggle/input/my_model/keras/default/2/my_model.h5"

# Load the model
model = load_model(model_path)

print("Model loaded successfully!")


# Example from dataset

CLASSIFICATION CODE

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import cv2

# Assuming your model is already trained and compiled
# If not, you can load a saved model using tf.keras.models.load_model('your_model_path')

# Function to preprocess the image
def get_img_array(img_path, size):
    # Load the image with the target size
    img = image.load_img(img_path, target_size=size)
    # Convert the image to a numpy array
    array = image.img_to_array(img)
    # Expand dimensions to create a batch of size 1
    array = np.expand_dims(array, axis=0)
    return array

# Path to the image you want to visualize
img_path = '/kaggle/input/ultrasound-breast-images-for-breast-cancer/ultrasound breast classification/val/malignant/malignant (1)-rotated1.png'  # Replace with your image path

# Prepare the image
img_array = get_img_array(img_path, size=(img_size, img_size))

In [ ]:
# Predict the class of the image
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions[0])
class_names = ['benign', 'malignant']  # Ensure these match your dataset's class names
predicted_label = class_names[predicted_class]

print(f"The model predicts this image is: {predicted_label}")

PERCENTAGE AND HEATMAP CODE

In [ ]:
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load the trained model
model_path = "/kaggle/input/my_model/keras/default/2/my_model.h5"
model = load_model(model_path)

# Function to preprocess input image
def preprocess_image(img_path, img_size=224):
    img = image.load_img(img_path, target_size=(img_size, img_size))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array, img

# Function to generate Grad-CAM heatmap
def get_grad_cam(model, img_array, layer_name):
    grad_model = tf.keras.models.Model(
        inputs=model.input, 
        outputs=[model.get_layer(layer_name).output, model.output]
    )
    
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        class_idx = np.argmax(predictions[0])
        loss = predictions[:, class_idx]
    
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0] * pooled_grads
    heatmap = np.mean(conv_outputs, axis=-1)
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

# Function to overlay heatmap on image
def overlay_heatmap(img, heatmap, alpha=0.4):
    heatmap = cv2.resize(heatmap, (img.size[0], img.size[1]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    img = np.array(img)
    overlay = cv2.addWeighted(img, 1 - alpha, heatmap, alpha, 0)
    return overlay

# Function to calculate affected percentage
def calculate_affected_percentage(heatmap, threshold=0.5):
    affected_area = np.sum(heatmap > threshold)
    total_area = heatmap.size
    affected_percentage = (affected_area / total_area) * 100
    return affected_percentage

# Load and process image
img_path = "/kaggle/input/ultrasound-breast-images-for-breast-cancer/ultrasound breast classification/val/malignant/malignant (1)-rotated1.png"
img_array, img = preprocess_image(img_path, img_size=224)

# Generate Grad-CAM heatmap
layer_name = "block5_conv3"  # Adjust based on your model architecture
heatmap = get_grad_cam(model, img_array, layer_name)

# Calculate affected percentage
affected_percentage = calculate_affected_percentage(heatmap)

# Overlay heatmap
overlayed_img = overlay_heatmap(img, heatmap)

# Display results
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
axes[0].imshow(img)
axes[0].set_title("Original Image")
axes[1].imshow(overlayed_img)
axes[1].set_title(f"Grad-CAM Heatmap\nAffected Area: {affected_percentage:.2f}%")
plt.show()


# Example from outside image

CLASSIFICATION CODE

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import cv2

# Assuming your model is already trained and compiled
# If not, you can load a saved model using tf.keras.models.load_model('your_model_path')

# Function to preprocess the image
def get_img_array(img_path, size):
    # Load the image with the target size
    img = image.load_img(img_path, target_size=size)
    # Convert the image to a numpy array
    array = image.img_to_array(img)
    # Expand dimensions to create a batch of size 1
    array = np.expand_dims(array, axis=0)
    return array

# Path to the image you want to visualize
img_path = '/kaggle/input/cancer2/Screenshot 2025-02-16 215943.png'  # Replace with your image path

# Prepare the image
img_array = get_img_array(img_path, size=(img_size, img_size))

In [ ]:
# Predict the class of the image
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions[0])
class_names = ['benign', 'malignant']  # Ensure these match your dataset's class names
predicted_label = class_names[predicted_class]

print(f"The model predicts this image is: {predicted_label}")

PERCENTAGE AND HEATMAP CODE

In [ ]:
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load the trained model
model_path = "/kaggle/input/my_model/keras/default/2/my_model.h5"
model = load_model(model_path)

# Function to preprocess input image
def preprocess_image(img_path, img_size=224):
    img = image.load_img(img_path, target_size=(img_size, img_size))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array, img

# Function to generate Grad-CAM heatmap
def get_grad_cam(model, img_array, layer_name):
    grad_model = tf.keras.models.Model(
        inputs=model.input, 
        outputs=[model.get_layer(layer_name).output, model.output]
    )
    
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        class_idx = np.argmax(predictions[0])
        loss = predictions[:, class_idx]
    
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0] * pooled_grads
    heatmap = np.mean(conv_outputs, axis=-1)
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

# Function to overlay heatmap on image
def overlay_heatmap(img, heatmap, alpha=0.4):
    heatmap = cv2.resize(heatmap, (img.size[0], img.size[1]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    img = np.array(img)
    overlay = cv2.addWeighted(img, 1 - alpha, heatmap, alpha, 0)
    return overlay

# Function to calculate affected percentage
def calculate_affected_percentage(heatmap, threshold=0.5):
    affected_area = np.sum(heatmap > threshold)
    total_area = heatmap.size
    affected_percentage = (affected_area / total_area) * 100
    return affected_percentage

# Load and process image
img_path = "/kaggle/input/cancer2/Screenshot 2025-02-16 215943.png"
img_array, img = preprocess_image(img_path, img_size=224)

# Generate Grad-CAM heatmap
layer_name = "block5_conv3"  # Adjust based on your model architecture
heatmap = get_grad_cam(model, img_array, layer_name)

# Calculate affected percentage
affected_percentage = calculate_affected_percentage(heatmap)

# Overlay heatmap
overlayed_img = overlay_heatmap(img, heatmap)

# Display results
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
axes[0].imshow(img)
axes[0].set_title("Original Image")
axes[1].imshow(overlayed_img)
axes[1].set_title(f"Grad-CAM Heatmap\nAffected Area: {affected_percentage:.2f}%")
plt.show()


# Example from outside image with benign

CLASSIFICATION CODE

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import cv2

# Assuming your model is already trained and compiled
# If not, you can load a saved model using tf.keras.models.load_model('your_model_path')

# Function to preprocess the image
def get_img_array(img_path, size):
    # Load the image with the target size
    img = image.load_img(img_path, target_size=size)
    # Convert the image to a numpy array
    array = image.img_to_array(img)
    # Expand dimensions to create a batch of size 1
    array = np.expand_dims(array, axis=0)
    return array

# Path to the image you want to visualize
img_path = '/kaggle/input/cancer4/Screenshot 2025-02-16 214605.png'  # Replace with your image path

# Prepare the image
img_array = get_img_array(img_path, size=(img_size, img_size))

In [ ]:
# Predict the class of the image
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions[0])
class_names = ['benign', 'malignant']  # Ensure these match your dataset's class names
predicted_label = class_names[predicted_class]

print(f"The model predicts this image is: {predicted_label}")